In [15]:
import numpy as np
import torch
import math
!pip install gpytorch
import gpytorch
from gpytorch.kernels.rbf_kernel import postprocess_rbf
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from collections import OrderedDict
import copy

In [4]:

def function(X,noise=0):
  def func(x):
    return (math.pow((x[1]-5.1/(4*math.pow(3.14,2))*math.pow(x[0],2)+5/3.14*x[0]-6),2)+10*(1-1/(8*3.14))*math.cos(x[0])+10)*math.sin(x[2]/5)
  return torch.tensor(np.apply_along_axis(func, 1, X).reshape(-1,1),dtype = torch.float32)

def get_proj_matrix(D,d,hypersphere=0):
  ''' Takes three arguments D : column size, d : row size and hypersphere = 0/1 (if hypersphere is 1 then sample each row of B from S^{D-1} else from N(0,1))
  returns a random B matrix of shape d x D '''
  A = np.random.normal(0,1,(d,D))
  if hypersphere==1:
    for row in range(d):
      N = np.linalg.norm(A[row,:])
      A[row,:] = A[row,:]/N
  return A


batch_shape = torch.Size([1])
a1 = torch.linspace(-5, 10, 10)
a2 = torch.linspace(0, 15, 10)
a3 = torch.linspace(-5,5,10)
train_x = torch.zeros((10*10*10,3),dtype=torch.float32)
ctr=0
for i1 in a1:
    for i2 in a2:
        for i3 in a3:
            train_x[ctr,0] = i1
            train_x[ctr,1] = i2
            train_x[ctr,2] = i3
            ctr+=1
B = torch.tensor(get_proj_matrix(3,2,1),dtype=torch.float32)
train_x_d = (train_x @ B.t())
train_y = function(train_x)
train_y = train_y.squeeze(1)

class AleboKernel(gpytorch.kernels.Kernel):
  ''' creates instances of alebo kernel in which distance metric is mahalonobis instead of euclidean distance.'''
  def __init__(self,B):
    super().__init__(has_length=False,ard_num_dims=None,eps=0.0)
    self.d = B.shape[0]
    self.D = B.shape[1]
    self.B = B
    self.Binv = torch.pinverse(B)
    # self.dtype = B.dtype
    # self.batch_shape = batch_shape

    A = torch.qr(torch.randn(self.D,self.D))[0]
    ABinv = A[:self.d,:] @ self.Binv
    T = ABinv.t() @ ABinv
    U = torch.cholesky(T,upper = True)
    self.idx = U.nonzero().t().tolist()
    Uvec = U[self.idx]#.repeat(*batch_shape,1)
    print(Uvec)
    self.register_parameter(name = "Uvec",parameter=torch.nn.Parameter(Uvec))
    
  def forward(self,x1,x2,**params):
    U_t = torch.zeros(self.Uvec.shape[:-1]+torch.Size([self.d,self.d]))
    U_t[...,self.idx[1],self.idx[0]] = self.Uvec

    z1 = x1 @ U_t
    z2 = x2 @ U_t

    return self.covar_dist(z1,z2,square_dist=True,dist_postprocess_func=postprocess_rbf,**params)

class AleboGP(gpytorch.models.ExactGP):
  ''' Creates instance of Alebo GP by taking into account the mahalonobis kernel '''
  def __init__(self, train_x, train_y, likelihood,B):
    super().__init__(train_x, train_y, likelihood)
    self.covar_module = gpytorch.kernels.ScaleKernel(base_kernel=AleboKernel(B=B))
    self.mean_module = gpytorch.means.ConstantMean()
  def forward(self,x):
    mean_x = self.mean_module(x)
    covar_x = self.covar_module(x)
    return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = AleboGP(train_x_d, train_y, likelihood,B)



tensor([1.0306, 0.2859, 0.9611])


In [ ]:

def train(model,likelihood,train_x,train_y,training_iter=2000):
  ''' Takes model and training data and return the optimised parameter of the AleboGP model'''
# training_iter = 50
  model.train()
  likelihood.train()

  optimizer = torch.optim.Adam(model.parameters(), lr=0.1) 

  mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

  for i in range(training_iter):
      optimizer.zero_grad()
      output = model(train_x)
  
      loss = -mll(output, train_y).sum()
      loss.backward()
      optimizer.step()
      params_dict = OrderedDict(mll.named_parameters())
      print(loss)
  
  model.eval()
  likelihood.eval()

  return model,likelihood

# model,likelihood = train(model,likelihood,train_x_d,train_y)

In [ ]:
def get_r2_square(model,likelihood,train_x,train_y):
  ''' takes mode and training data and calculates r2 square score for of the model on the data'''
  observed_pred = likelihood(model(train_x))
  lower,upper = observed_pred.confidence_region()
  r2_square = 0
  sum_diff = 0
  for i in range(train_x.shape[0]):
    r2_square += (observed_pred.mean[i].detach().numpy()-train_y[i].numpy())**2
  sum_diff += np.sum(upper.detach().numpy())-np.sum(lower.detach().numpy())

  return r2_square,sum_diff
    
def get_best_fit_gp(train_x,train_y,n_trials=1):
  ''' generates and train n_trials number of models and return the model that fits the data in the best way by using r2 square metric. '''
  ## Use MLL if it gives poor result.
  best_state = {}
  r2_square = 1e9
  for i in range(n_trials):
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = AleboGP(train_x, train_y, likelihood,B)

    model,likelihood = train(model,likelihood,train_x,train_y)

    r2_square_i,sum_diff_i = get_r2_square(model,likelihood,train_x,train_y)

    if r2_square_i < r2_square:
      best_state = model.state_dict()
      r2_square = r2_square_i
  
  best_likelihood =gpytorch.likelihoods.GaussianLikelihood()
  best_model = AleboGP(train_x, train_y, likelihood,B)
  best_model.state_dict(best_state)

  return best_likelihood,best_model


# def sample_U(likelihood,model,nsamp=10):
  # return list of Uvec for each model
likelihood, model = get_best_fit_gp(train_x @ B.t(),train_y)



In [ ]:
def get_param_hessian(model,likelihood,x0):
  ## x0 is the model parameter at which we have to return  grad

  model.train()
  likelihood.train()

  mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
  mll = set_param(mll,x0)
  Uvev_0 = np.array([])
  for x in mll.named_parameters():
    param_name = x[0]
    # param_grad = 
  
  

In [ ]:
model.eval()
likelihood.eval()

observed_pred = likelihood(model(train_x @ B.t()))
lower,upper = observed_pred.confidence_region()


plt.plot(train_y, observed_pred.mean.detach().numpy(),'bo' ,label="Pred")
plt.plot(train_y,train_y,label='Ideal')
plt.legend()

[tensor([-9.5011], grad_fn=<ViewBackward>)]


AttributeError: ignored

In [ ]:
def double_derivative(i,epsilon_i,m1,m2,l1,l2,train_x,train_y,training_iter):
    '''Function which takes the models m1 and m2 (m2 is for m1+epsilon)
        and returns the double derivative for i-th component'''
    m1,l1 = train(m1,l1,train_x,train_y,training_iter)
    m2,l2 = train(m2,l2,train_x,train_y,training_iter)
    g1 = (m1.covar_module.base_kernel.Uvec.grad)
    g2 = (m2.covar_module.base_kernel.Uvec.grad)
    return (g2[i] - g1[i])/epsilon_i


def print_param(model):
  print()
  for x in model.named_parameters():
      print(x)
  print()

def update_Uvec(model,likelihood,i,ep):
  ''' updates the ith element of Uvec model parameter from Uvec[i] to Uvec[i]+ep and returns the model with updated parameters'''
  model_copy = copy.deepcopy(model)
  print_param(model_copy)
  Uvec = model_copy.covar_module.base_kernel.Uvec
  model_copy.covar_module.base_kernel.Uvec.requires_grad_(False)
  Uvec[i] = Uvec[i]+ep
  model_copy.covar_module.base_kernel.Uvec.requires_grad_(True)
  print_param(model_copy)
update_Uvec(model,likelihood,0,torch.tensor(1))




('likelihood.noise_covar.raw_noise', Parameter containing:
tensor([37.5372], requires_grad=True))
('covar_module.raw_outputscale', Parameter containing:
tensor(0., requires_grad=True))
('covar_module.base_kernel.Uvec', Parameter containing:
tensor([ 0.9627, -0.3541,  0.5022], requires_grad=True))
('mean_module.constant', Parameter containing:
tensor([0.], requires_grad=True))


('likelihood.noise_covar.raw_noise', Parameter containing:
tensor([37.5372], requires_grad=True))
('covar_module.raw_outputscale', Parameter containing:
tensor(0., requires_grad=True))
('covar_module.base_kernel.Uvec', Parameter containing:
tensor([ 1.9627, -0.3541,  0.5022], requires_grad=True))
('mean_module.constant', Parameter containing:
tensor([0.], requires_grad=True))



In [ ]:
def get_final_pred(models,likelihoods,test_x): # test_x is in n*d dimensions.returns means,variances
    preds = [likelihood(model(test_x @ B.t())) for likelihood,model in zip(likelihoods,models)]
    means = torch.tensor([[y.mean for y in x] for x in preds]).t()
    variances = torch.tensor([[y.variance for y in x] for x in preds]).t()
    mu = torch.mean(means,axis=1)
    variance = torch.mean(variances,axis=1) + torch.var(means,axis=1,unbiased=False)
    return mu,variance

get_final_pred([model],[likelihood],train_x[0:2,:])


(tensor([-9.5011, -5.9215]), tensor([1.1790, 1.1869]))

In [20]:
def constrained_minimize(objective,Binv,x0):
    def constraint_generator_minus_1(Binv,i):
        def con(x):
            x_D = Binv @ x
            return x_D[i]+1
        return con
    def constraint_generator_plus_1(Binv,i):
        def con(x):
            x_D = Binv @ x
            return 1-x_D[i]
        return con
    constraints_fun = []
    for i in range(B.shape[0]):
        constraints_fun.append(constraint_generator_minus_1(Binv,i))
        constraints_fun.append(constraint_generator_plus_1(Binv,i))
    constraints = [{'type':'ineq', 'fun':fun} for fun in constraints_fun]
    print(constraints)
    candidate = minimize(objective,x0,constraints=constraints,method='SLSQP')
            


def objective(x):
    print("x=",x)
    return x.sum()
B = np.eye(3)
x0 = np.array([0.5,0.5,0.5])
constrained_minimize(objective,B,x0)



[{'type': 'ineq', 'fun': <function constrained_minimize.<locals>.constraint_generator_minus_1.<locals>.con at 0x7f64ae60edd0>}, {'type': 'ineq', 'fun': <function constrained_minimize.<locals>.constraint_generator_plus_1.<locals>.con at 0x7f64ae60eb90>}, {'type': 'ineq', 'fun': <function constrained_minimize.<locals>.constraint_generator_minus_1.<locals>.con at 0x7f64ae60ea70>}, {'type': 'ineq', 'fun': <function constrained_minimize.<locals>.constraint_generator_plus_1.<locals>.con at 0x7f64ae60e4d0>}, {'type': 'ineq', 'fun': <function constrained_minimize.<locals>.constraint_generator_minus_1.<locals>.con at 0x7f64ae60eb00>}, {'type': 'ineq', 'fun': <function constrained_minimize.<locals>.constraint_generator_plus_1.<locals>.con at 0x7f64ae60e440>}]
x= [0.5 0.5 0.5]
x= [0.5 0.5 0.5]
x= [0.50000001 0.5        0.5       ]
x= [0.5        0.50000001 0.5       ]
x= [0.5        0.5        0.50000001]
x= [-0.5 -0.5 -0.5]
x= [-0.5 -0.5 -0.5]
x= [-0.49999999 -0.5        -0.5       ]
x= [-0.5   